# Trying to use time features

In [1]:
import pandas as pd

df = pd.read_csv('Whole_dataset1.txt')

df = df.sort_values(by='Date')

import gc; gc.collect()

0

## Creating a new feature 'timestamp'

In [2]:
date = df.Date.tolist()
timestamp = [int(''.join(i.split('-'))) for i in date]
df['timestamp'] = timestamp; del timestamp
del date

## Doing joins

In [3]:
df = df.drop(axis=1,labels=['Date'])

ddf = pd.concat([pd.read_pickle('info_1.pkl'),
          pd.read_pickle('info_2.pkl'),
          pd.read_pickle('info_3.pkl')])

df = df.join(ddf.set_index('MovieID'),on='MovieID')
del ddf

ddf = pd.concat([pd.read_pickle('movie_info1.pkl'),
          pd.read_pickle('movie_info2.pkl'),
          pd.read_pickle('movie_info3.pkl'),
                pd.read_pickle('movie_info4.pkl'),
                pd.read_pickle('movie_info5.pkl')])

df = df.join(ddf.set_index('CustomerID'),on='CustomerID')
del ddf

ddf = pd.read_pickle('modified_movie_title_year.pkl')

df = df.join(ddf.set_index('MovieID'),on='MovieID')
del ddf

df = df.drop(axis=1, labels=['MovieID','CustomerID'])

In [4]:
from sklearn.metrics import mean_squared_error
import math
rmse = []

### Logistic regression with simple time features

In [5]:
'''Printing a sample from the traindata to see the features that have been used'''
df.head(3)

,Score,timestamp,NumberOfRating,MeanRating,StdDev,Cust_mean,cust_stddev,freq_of_rating,Year_of_release
9056171,5,19991111,113377,3.971317,0.893782,3.364807,0.958341,233,19870101
14892677,3,19991111,22415,3.439929,1.099609,3.364807,0.958341,233,19990101
19585852,4,19991111,40191,3.952726,0.888977,3.364807,0.958341,233,19980101


In [6]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs=-1,normalize=True)
l = int(df.shape[0]*.7)
reg.fit(df.drop(axis=1,labels=['Score'])[:l],df.Score[:l])
hypothesis = reg.predict(df.drop(axis=1,labels=['Score'])[l:])

In [7]:
e = math.sqrt(mean_squared_error(df.Score[l:],hypothesis))
e

0.9285131697373992

In [8]:
rmse.append(e)

### Using SGD Regressor

In [10]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor()
l = int(df.shape[0]*.7)
reg.fit(df.drop(axis=1,labels=['Score'])[:l],df.Score[:l])
hypothesis = reg.predict(df.drop(axis=1,labels=['Score'])[l:])

In [11]:
e = math.sqrt(mean_squared_error(df.Score[l:],hypothesis))
e

4.055276644453863e+22

In [12]:
print (hypothesis[:10])

[  4.04965655e+22   4.05150541e+22   4.05797365e+22   4.05797385e+22
   4.05154135e+22   4.04669849e+22   4.04806648e+22   4.04806622e+22
   4.04918726e+22   4.04675011e+22]


    Something is going wrong with SGD Regressor

    There isnt't much increase in the performance. Let's consider only the data from 2004 and onwards

In [13]:
df_2004 = df[df.timestamp > 20040000]
df_2004.shape

(20131855, 9)

In [14]:
reg = LinearRegression(n_jobs=-1,normalize=True)
l = int(df_2004.shape[0]*.7)
reg.fit(df_2004.drop(axis=1,labels=['Score'])[:l],df_2004.Score[:l])
hypothesis = reg.predict(df_2004.drop(axis=1,labels=['Score'])[l:])

In [15]:
e = math.sqrt(mean_squared_error(df_2004.Score[l:],hypothesis))
del df_2004
e

0.9266706645591626

    Let's see the same on only 2005 data

In [16]:
df_2005 = df[df.timestamp > 20050000]
df_2005.shape

(12562227, 9)

In [17]:
reg = LinearRegression(n_jobs=-1,normalize=True)
l = int(df_2005.shape[0]*.7)
reg.fit(df_2005.drop(axis=1,labels=['Score'])[:l],df_2005.Score[:l])
hypothesis = reg.predict(df_2005.drop(axis=1,labels=['Score'])[l:])

In [18]:
e = math.sqrt(mean_squared_error(df_2005.Score[l:],hypothesis))
del df_2005
e

0.9213388518627924

In [19]:
del df